# MINOR PROJECT -- Cricket Score Predictor (using Machine Learning)
## 2) Feature Extraction

In [1]:
import pandas as pd
import pickle
import numpy as np

/home/nakshatra/.local/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# (1) For t20's

In [2]:
## loading the dataframe in df from level 2 pickle file
df = pickle.load(open('dataset_level2.pkl','rb'))

In [396]:
df

# batting_team
# bowling team
# city
# current score
# ball left
# wickets left
# current rr
# last five

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,0,Sri Lanka,India,0.1,2,None,Indore,Holkar Cricket Stadium
1,0,Sri Lanka,India,0.2,4,None,Indore,Holkar Cricket Stadium
2,0,Sri Lanka,India,0.3,0,None,Indore,Holkar Cricket Stadium
3,0,Sri Lanka,India,0.4,1,None,Indore,Holkar Cricket Stadium
4,0,Sri Lanka,India,0.5,0,None,Indore,Holkar Cricket Stadium
...,...,...,...,...,...,...,...,...
116542,1431,New Zealand,Pakistan,17.5,1,None,London,Kennington Oval
116543,1431,New Zealand,Pakistan,17.6,1,None,London,Kennington Oval
116544,1431,New Zealand,Pakistan,18.1,4,None,London,Kennington Oval
116545,1431,New Zealand,Pakistan,18.2,0,None,London,Kennington Oval


In [397]:
## counting null values
df.isnull().sum()

match_id                0
batting_team            0
bowling_team            0
ball                    0
runs                    0
player_dismissed    61303
city                 8671
venue                   0
dtype: int64

In [398]:
## Finding the venues of the rows with missing cities.
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        3092
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sharjah Cricket Stadium                     249
Sylhet International Cricket Stadium        128
Carrara Oval                                 64
Name: count, dtype: int64

In [399]:
## Using Numpy to use first word of stadium as city
cities = np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])

In [400]:
df['city'] = cities

In [401]:
df.isnull().sum()

match_id                0
batting_team            0
bowling_team            0
ball                    0
runs                    0
player_dismissed    61303
city                    0
venue                   0
dtype: int64

In [402]:
## making playre_dismissed value to 0 where there in None
df['player_dismissed'] = df['player_dismissed'].apply(lambda x: 0 if x == None else 1)


In [403]:
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [404]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,0,Sri Lanka,India,0.1,2,0,Indore,Holkar Cricket Stadium
1,0,Sri Lanka,India,0.2,4,0,Indore,Holkar Cricket Stadium
2,0,Sri Lanka,India,0.3,0,0,Indore,Holkar Cricket Stadium
3,0,Sri Lanka,India,0.4,1,0,Indore,Holkar Cricket Stadium
4,0,Sri Lanka,India,0.5,0,0,Indore,Holkar Cricket Stadium
...,...,...,...,...,...,...,...,...
116542,1431,New Zealand,Pakistan,17.5,1,0,London,Kennington Oval
116543,1431,New Zealand,Pakistan,17.6,1,0,London,Kennington Oval
116544,1431,New Zealand,Pakistan,18.1,4,0,London,Kennington Oval
116545,1431,New Zealand,Pakistan,18.2,0,0,London,Kennington Oval


In [405]:
## Out of so many cities some cities hosted only 1 or 2 matches so we will not considee them
## We will take those cities where at least 5 matches ie 600 balls were bowled
eligible_cities = df['city'].value_counts()[df['city'].value_counts() > 600].index.tolist()

In [406]:
df = df[df['city'].isin(eligible_cities)]

In [407]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
372,3,Sri Lanka,Pakistan,0.1,1,0,Abu Dhabi,Sheikh Zayed Stadium
373,3,Sri Lanka,Pakistan,0.2,0,0,Abu Dhabi,Sheikh Zayed Stadium
374,3,Sri Lanka,Pakistan,0.3,0,1,Abu Dhabi,Sheikh Zayed Stadium
375,3,Sri Lanka,Pakistan,0.4,2,0,Abu Dhabi,Sheikh Zayed Stadium
376,3,Sri Lanka,Pakistan,0.5,1,0,Abu Dhabi,Sheikh Zayed Stadium
...,...,...,...,...,...,...,...,...
116542,1431,New Zealand,Pakistan,17.5,1,0,London,Kennington Oval
116543,1431,New Zealand,Pakistan,17.6,1,0,London,Kennington Oval
116544,1431,New Zealand,Pakistan,18.1,4,0,London,Kennington Oval
116545,1431,New Zealand,Pakistan,18.2,0,0,London,Kennington Oval


In [408]:
## Creating a current score column by doing cumulative sum in runs column
# Convert 'runs' column to numeric dtype
df['runs'] = pd.to_numeric(df['runs'], errors='coerce')

# Create a current score column by doing cumulative sum in the 'runs' column
df['current_score'] = df.groupby('match_id')['runs'].cumsum()

/tmp/ipykernel_5298/592937206.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['runs'] = pd.to_numeric(df['runs'], errors='coerce')
/tmp/ipykernel_5298/592937206.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['current_score'] = df.groupby('match_id')['runs'].cumsum()


In [409]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score
372,3,Sri Lanka,Pakistan,0.1,1,0,Abu Dhabi,Sheikh Zayed Stadium,1
373,3,Sri Lanka,Pakistan,0.2,0,0,Abu Dhabi,Sheikh Zayed Stadium,1
374,3,Sri Lanka,Pakistan,0.3,0,1,Abu Dhabi,Sheikh Zayed Stadium,1
375,3,Sri Lanka,Pakistan,0.4,2,0,Abu Dhabi,Sheikh Zayed Stadium,3
376,3,Sri Lanka,Pakistan,0.5,1,0,Abu Dhabi,Sheikh Zayed Stadium,4
...,...,...,...,...,...,...,...,...,...
116542,1431,New Zealand,Pakistan,17.5,1,0,London,Kennington Oval,94
116543,1431,New Zealand,Pakistan,17.6,1,0,London,Kennington Oval,95
116544,1431,New Zealand,Pakistan,18.1,4,0,London,Kennington Oval,99
116545,1431,New Zealand,Pakistan,18.2,0,0,London,Kennington Oval,99


In [410]:
## making ball_no and over column using ball column
df['over'] = df['ball'].apply(lambda x:str(x).split(".")[0])
df['ball_no'] = df['ball'].apply(lambda x:str(x).split(".")[1])
df

/tmp/ipykernel_5298/3273211203.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['over'] = df['ball'].apply(lambda x:str(x).split(".")[0])
/tmp/ipykernel_5298/3273211203.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ball_no'] = df['ball'].apply(lambda x:str(x).split(".")[1])


,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score,over,ball_no
372,3,Sri Lanka,Pakistan,0.1,1,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,1
373,3,Sri Lanka,Pakistan,0.2,0,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,2
374,3,Sri Lanka,Pakistan,0.3,0,1,Abu Dhabi,Sheikh Zayed Stadium,1,0,3
375,3,Sri Lanka,Pakistan,0.4,2,0,Abu Dhabi,Sheikh Zayed Stadium,3,0,4
376,3,Sri Lanka,Pakistan,0.5,1,0,Abu Dhabi,Sheikh Zayed Stadium,4,0,5
...,...,...,...,...,...,...,...,...,...,...,...
116542,1431,New Zealand,Pakistan,17.5,1,0,London,Kennington Oval,94,17,5
116543,1431,New Zealand,Pakistan,17.6,1,0,London,Kennington Oval,95,17,6
116544,1431,New Zealand,Pakistan,18.1,4,0,London,Kennington Oval,99,18,1
116545,1431,New Zealand,Pakistan,18.2,0,0,London,Kennington Oval,99,18,2


In [411]:
## making balls_bowled column using ball_no and over column
df['balls_bowled'] = (df['over'].astype('int')*6) + df['ball_no'].astype('int')
df

/tmp/ipykernel_5298/1370017395.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_bowled'] = (df['over'].astype('int')*6) + df['ball_no'].astype('int')


,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score,over,ball_no,balls_bowled
372,3,Sri Lanka,Pakistan,0.1,1,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,1,1
373,3,Sri Lanka,Pakistan,0.2,0,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,2,2
374,3,Sri Lanka,Pakistan,0.3,0,1,Abu Dhabi,Sheikh Zayed Stadium,1,0,3,3
375,3,Sri Lanka,Pakistan,0.4,2,0,Abu Dhabi,Sheikh Zayed Stadium,3,0,4,4
376,3,Sri Lanka,Pakistan,0.5,1,0,Abu Dhabi,Sheikh Zayed Stadium,4,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...
116542,1431,New Zealand,Pakistan,17.5,1,0,London,Kennington Oval,94,17,5,107
116543,1431,New Zealand,Pakistan,17.6,1,0,London,Kennington Oval,95,17,6,108
116544,1431,New Zealand,Pakistan,18.1,4,0,London,Kennington Oval,99,18,1,109
116545,1431,New Zealand,Pakistan,18.2,0,0,London,Kennington Oval,99,18,2,110


In [412]:
## making balls_left column
df['balls_left'] = 120 - df['balls_bowled']
df['balls_left'] = df['balls_left'].apply(lambda x:0 if x<0 else x)
df

/tmp/ipykernel_5298/2709722656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_left'] = 120 - df['balls_bowled']
/tmp/ipykernel_5298/2709722656.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['balls_left'] = df['balls_left'].apply(lambda x:0 if x<0 else x)


,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score,over,ball_no,balls_bowled,balls_left
372,3,Sri Lanka,Pakistan,0.1,1,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,1,1,119
373,3,Sri Lanka,Pakistan,0.2,0,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,2,2,118
374,3,Sri Lanka,Pakistan,0.3,0,1,Abu Dhabi,Sheikh Zayed Stadium,1,0,3,3,117
375,3,Sri Lanka,Pakistan,0.4,2,0,Abu Dhabi,Sheikh Zayed Stadium,3,0,4,4,116
376,3,Sri Lanka,Pakistan,0.5,1,0,Abu Dhabi,Sheikh Zayed Stadium,4,0,5,5,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116542,1431,New Zealand,Pakistan,17.5,1,0,London,Kennington Oval,94,17,5,107,13
116543,1431,New Zealand,Pakistan,17.6,1,0,London,Kennington Oval,95,17,6,108,12
116544,1431,New Zealand,Pakistan,18.1,4,0,London,Kennington Oval,99,18,1,109,11
116545,1431,New Zealand,Pakistan,18.2,0,0,London,Kennington Oval,99,18,2,110,10


In [413]:
## making wickets left using player_dismissed column
df['player_dismissed'] = df['player_dismissed'].astype('int')
df

/tmp/ipykernel_5298/3596514388.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_dismissed'] = df['player_dismissed'].astype('int')


,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score,over,ball_no,balls_bowled,balls_left
372,3,Sri Lanka,Pakistan,0.1,1,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,1,1,119
373,3,Sri Lanka,Pakistan,0.2,0,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,2,2,118
374,3,Sri Lanka,Pakistan,0.3,0,1,Abu Dhabi,Sheikh Zayed Stadium,1,0,3,3,117
375,3,Sri Lanka,Pakistan,0.4,2,0,Abu Dhabi,Sheikh Zayed Stadium,3,0,4,4,116
376,3,Sri Lanka,Pakistan,0.5,1,0,Abu Dhabi,Sheikh Zayed Stadium,4,0,5,5,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116542,1431,New Zealand,Pakistan,17.5,1,0,London,Kennington Oval,94,17,5,107,13
116543,1431,New Zealand,Pakistan,17.6,1,0,London,Kennington Oval,95,17,6,108,12
116544,1431,New Zealand,Pakistan,18.1,4,0,London,Kennington Oval,99,18,1,109,11
116545,1431,New Zealand,Pakistan,18.2,0,0,London,Kennington Oval,99,18,2,110,10


In [414]:
# Group by 'match_id' and calculate cumulative sum
df['wickets_left'] = df.groupby('match_id')['player_dismissed'].cumsum()
df['wickets_left'] = 10 - df['player_dismissed']

/tmp/ipykernel_5298/2373859241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wickets_left'] = df.groupby('match_id')['player_dismissed'].cumsum()
/tmp/ipykernel_5298/2373859241.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wickets_left'] = 10 - df['player_dismissed']


In [415]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score,over,ball_no,balls_bowled,balls_left,wickets_left
372,3,Sri Lanka,Pakistan,0.1,1,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,1,1,119,10
373,3,Sri Lanka,Pakistan,0.2,0,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,2,2,118,10
374,3,Sri Lanka,Pakistan,0.3,0,1,Abu Dhabi,Sheikh Zayed Stadium,1,0,3,3,117,9
375,3,Sri Lanka,Pakistan,0.4,2,0,Abu Dhabi,Sheikh Zayed Stadium,3,0,4,4,116,10
376,3,Sri Lanka,Pakistan,0.5,1,0,Abu Dhabi,Sheikh Zayed Stadium,4,0,5,5,115,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116542,1431,New Zealand,Pakistan,17.5,1,0,London,Kennington Oval,94,17,5,107,13,10
116543,1431,New Zealand,Pakistan,17.6,1,0,London,Kennington Oval,95,17,6,108,12,10
116544,1431,New Zealand,Pakistan,18.1,4,0,London,Kennington Oval,99,18,1,109,11,10
116545,1431,New Zealand,Pakistan,18.2,0,0,London,Kennington Oval,99,18,2,110,10,10


In [416]:
##Calculating current run rate
df['crr'] = (df['current_score']*6)/df['balls_bowled']

/tmp/ipykernel_5298/562408054.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['crr'] = (df['current_score']*6)/df['balls_bowled']


In [417]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr
372,3,Sri Lanka,Pakistan,0.1,1,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,1,1,119,10,6.000000
373,3,Sri Lanka,Pakistan,0.2,0,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,2,2,118,10,3.000000
374,3,Sri Lanka,Pakistan,0.3,0,1,Abu Dhabi,Sheikh Zayed Stadium,1,0,3,3,117,9,2.000000
375,3,Sri Lanka,Pakistan,0.4,2,0,Abu Dhabi,Sheikh Zayed Stadium,3,0,4,4,116,10,4.500000
376,3,Sri Lanka,Pakistan,0.5,1,0,Abu Dhabi,Sheikh Zayed Stadium,4,0,5,5,115,10,4.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116542,1431,New Zealand,Pakistan,17.5,1,0,London,Kennington Oval,94,17,5,107,13,10,5.271028
116543,1431,New Zealand,Pakistan,17.6,1,0,London,Kennington Oval,95,17,6,108,12,10,5.277778
116544,1431,New Zealand,Pakistan,18.1,4,0,London,Kennington Oval,99,18,1,109,11,10,5.449541
116545,1431,New Zealand,Pakistan,18.2,0,0,London,Kennington Oval,99,18,2,110,10,10,5.400000


In [418]:
print(df['runs'].unique())
print(df['runs'].isnull().sum())
print(df['runs'].dtype)


[1 0 2 4 6 3 5 7]
0
int64


In [419]:
# Assuming df is your DataFrame containing the data
df['runs'] = pd.to_numeric(df['runs'], errors='coerce')

# Group by 'match_id'
groups = df.groupby('match_id')

# Compute rolling sum for each group with NaN values for the first 29 balls
last_five = []
for group_id, group_data in groups:
    rolling_sum = group_data['runs'].rolling(window=30, min_periods=1).sum()
    # Set NaN for 'last_five_overs_sum' if fewer than 30 balls are available
    if len(group_data) < 30:
        rolling_sum.iloc[-1] = np.nan
    last_five.extend(rolling_sum.values.tolist())

# Assign 'last_five_overs_sum' to the DataFrame
df['last_five'] = last_five

/tmp/ipykernel_5298/4146293879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['runs'] = pd.to_numeric(df['runs'], errors='coerce')
/tmp/ipykernel_5298/4146293879.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_five'] = last_five


In [420]:
df

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
372,3,Sri Lanka,Pakistan,0.1,1,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,1,1,119,10,6.000000,1.0
373,3,Sri Lanka,Pakistan,0.2,0,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,2,2,118,10,3.000000,1.0
374,3,Sri Lanka,Pakistan,0.3,0,1,Abu Dhabi,Sheikh Zayed Stadium,1,0,3,3,117,9,2.000000,1.0
375,3,Sri Lanka,Pakistan,0.4,2,0,Abu Dhabi,Sheikh Zayed Stadium,3,0,4,4,116,10,4.500000,3.0
376,3,Sri Lanka,Pakistan,0.5,1,0,Abu Dhabi,Sheikh Zayed Stadium,4,0,5,5,115,10,4.800000,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116542,1431,New Zealand,Pakistan,17.5,1,0,London,Kennington Oval,94,17,5,107,13,10,5.271028,20.0
116543,1431,New Zealand,Pakistan,17.6,1,0,London,Kennington Oval,95,17,6,108,12,10,5.277778,20.0
116544,1431,New Zealand,Pakistan,18.1,4,0,London,Kennington Oval,99,18,1,109,11,10,5.449541,20.0
116545,1431,New Zealand,Pakistan,18.2,0,0,London,Kennington Oval,99,18,2,110,10,10,5.400000,20.0


In [431]:
final_df = df.groupby('match_id').sum()['runs'].reset_index().merge(df,on='match_id')
final_df

,match_id,runs_x,batting_team,bowling_team,ball,runs_y,player_dismissed,city,venue,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
0,3,102,Sri Lanka,Pakistan,0.1,1,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,1,1,119,10,6.000000,1.0
1,3,102,Sri Lanka,Pakistan,0.2,0,0,Abu Dhabi,Sheikh Zayed Stadium,1,0,2,2,118,10,3.000000,1.0
2,3,102,Sri Lanka,Pakistan,0.3,0,1,Abu Dhabi,Sheikh Zayed Stadium,1,0,3,3,117,9,2.000000,1.0
3,3,102,Sri Lanka,Pakistan,0.4,2,0,Abu Dhabi,Sheikh Zayed Stadium,3,0,4,4,116,10,4.500000,3.0
4,3,102,Sri Lanka,Pakistan,0.5,1,0,Abu Dhabi,Sheikh Zayed Stadium,4,0,5,5,115,10,4.800000,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51104,1431,99,New Zealand,Pakistan,17.5,1,0,London,Kennington Oval,94,17,5,107,13,10,5.271028,20.0
51105,1431,99,New Zealand,Pakistan,17.6,1,0,London,Kennington Oval,95,17,6,108,12,10,5.277778,20.0
51106,1431,99,New Zealand,Pakistan,18.1,4,0,London,Kennington Oval,99,18,1,109,11,10,5.449541,20.0
51107,1431,99,New Zealand,Pakistan,18.2,0,0,London,Kennington Oval,99,18,2,110,10,10,5.400000,20.0


In [432]:
final_df=final_df[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x']]
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
0,Sri Lanka,Pakistan,Abu Dhabi,1,119,10,6.000000,1.0,102
1,Sri Lanka,Pakistan,Abu Dhabi,1,118,10,3.000000,1.0,102
2,Sri Lanka,Pakistan,Abu Dhabi,1,117,9,2.000000,1.0,102
3,Sri Lanka,Pakistan,Abu Dhabi,3,116,10,4.500000,3.0,102
4,Sri Lanka,Pakistan,Abu Dhabi,4,115,10,4.800000,4.0,102
...,...,...,...,...,...,...,...,...,...
51104,New Zealand,Pakistan,London,94,13,10,5.271028,20.0,99
51105,New Zealand,Pakistan,London,95,12,10,5.277778,20.0,99
51106,New Zealand,Pakistan,London,99,11,10,5.449541,20.0,99
51107,New Zealand,Pakistan,London,99,10,10,5.400000,20.0,99


In [433]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        3
runs_x           0
dtype: int64

In [438]:
final_df.dropna(inplace=True)

In [439]:
final_df.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
dtype: int64

In [440]:
final_df = final_df.sample(final_df.shape[0]) ## shuffeling to neglecting bias

In [441]:
final_df

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
13004,England,Australia,Melbourne,30,98,10,8.181818,30.0,137
21325,West Indies,South Africa,Johannesburg,70,39,10,5.185185,28.0,131
48570,New Zealand,South Africa,Durban,144,4,10,7.448276,33.0,153
42003,Pakistan,India,Mirpur,92,31,10,6.202247,42.0,130
32265,India,England,Kolkata,83,36,10,5.928571,22.0,120
...,...,...,...,...,...,...,...,...,...
29898,England,Pakistan,Dubai,64,67,10,7.245283,33.0,150
30515,England,Sri Lanka,Colombo,107,54,9,9.727273,47.0,187
14554,New Zealand,West Indies,Auckland,0,119,10,0.000000,0.0,189
34740,England,Australia,Southampton,83,47,10,6.821918,41.0,145


In [442]:
##Saving this dataframe final_df in pickle ie level 3 
pickle.dump(final_df,open('dataset_level3.pkl','wb'))

# (2) For ODI's

In [3]:
## loading the dataframe in df from level 2 pickle file
df_odi = pickle.load(open('dataset_level2_odi.pkl','rb'))

In [4]:
df_odi

# batting_team
# bowling team
# city
# current score
# ball left
# wickets left
# current rr
# last five

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,0,England,New Zealand,0.1,1,None,Christchurch,AMI Stadium
1,0,England,New Zealand,0.2,0,None,Christchurch,AMI Stadium
2,0,England,New Zealand,0.3,0,None,Christchurch,AMI Stadium
3,0,England,New Zealand,0.4,0,None,Christchurch,AMI Stadium
4,0,England,New Zealand,0.5,1,None,Christchurch,AMI Stadium
...,...,...,...,...,...,...,...,...
519641,1798,Pakistan,India,49.2,1,None,Gwalior,Captain Roop Singh Stadium
519642,1798,Pakistan,India,49.3,1,None,Gwalior,Captain Roop Singh Stadium
519643,1798,Pakistan,India,49.4,1,None,Gwalior,Captain Roop Singh Stadium
519644,1798,Pakistan,India,49.5,6,None,Gwalior,Captain Roop Singh Stadium


In [6]:
## counting null values
df_odi.isnull().sum()

match_id                 0
batting_team             0
bowling_team             0
ball                     0
runs                     0
player_dismissed    372971
city                 61543
venue                    0
dtype: int64

In [7]:
## Finding the venues of the rows with missing cities.
df_odi[df_odi['city'].isnull()]['venue'].value_counts()

venue
Sydney Cricket Ground                      10525
Rangiri Dambulla International Stadium     10051
Melbourne Cricket Ground                    8696
Adelaide Oval                               7725
Dubai International Cricket Stadium         6314
Pallekele International Cricket Stadium     6170
Sharjah Cricket Stadium                     4658
Harare Sports Club                          2432
Multan Cricket Stadium                      1199
Queenstown Events Centre                     909
Chittagong Divisional Stadium                900
Rawalpindi Cricket Stadium                   617
Perth Stadium                                526
Dubai Sports City Cricket Stadium            518
Sharjah Cricket Association Stadium          303
Name: count, dtype: int64

In [9]:
## Using Numpy to use first word of stadium as city
cities = np.where(df_odi['city'].isnull(),df_odi['venue'].str.split().apply(lambda x:x[0]),df_odi['city'])

In [10]:
df_odi['city'] = cities

In [11]:
df_odi.isnull().sum()

match_id                 0
batting_team             0
bowling_team             0
ball                     0
runs                     0
player_dismissed    372971
city                     0
venue                    0
dtype: int64

In [12]:
## making playre_dismissed value to 0 where there in None
df_odi['player_dismissed'] = df_odi['player_dismissed'].apply(lambda x: 0 if x == None else 1)

In [13]:
df_odi.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [14]:
df_odi

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,0,England,New Zealand,0.1,1,0,Christchurch,AMI Stadium
1,0,England,New Zealand,0.2,0,0,Christchurch,AMI Stadium
2,0,England,New Zealand,0.3,0,0,Christchurch,AMI Stadium
3,0,England,New Zealand,0.4,0,0,Christchurch,AMI Stadium
4,0,England,New Zealand,0.5,1,0,Christchurch,AMI Stadium
...,...,...,...,...,...,...,...,...
519641,1798,Pakistan,India,49.2,1,0,Gwalior,Captain Roop Singh Stadium
519642,1798,Pakistan,India,49.3,1,0,Gwalior,Captain Roop Singh Stadium
519643,1798,Pakistan,India,49.4,1,0,Gwalior,Captain Roop Singh Stadium
519644,1798,Pakistan,India,49.5,6,0,Gwalior,Captain Roop Singh Stadium


In [15]:
## Out of so many cities some cities hosted only 1 or 2 matches so we will not considee them
## We will take those cities where at least 5 matches ie 600 balls were bowled
eligible_cities = df_odi['city'].value_counts()[df_odi['city'].value_counts() > 600].index.tolist()

In [16]:
df_odi = df_odi[df_odi['city'].isin(eligible_cities)]

In [17]:
df_odi

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,0,England,New Zealand,0.1,1,0,Christchurch,AMI Stadium
1,0,England,New Zealand,0.2,0,0,Christchurch,AMI Stadium
2,0,England,New Zealand,0.3,0,0,Christchurch,AMI Stadium
3,0,England,New Zealand,0.4,0,0,Christchurch,AMI Stadium
4,0,England,New Zealand,0.5,1,0,Christchurch,AMI Stadium
...,...,...,...,...,...,...,...,...
519641,1798,Pakistan,India,49.2,1,0,Gwalior,Captain Roop Singh Stadium
519642,1798,Pakistan,India,49.3,1,0,Gwalior,Captain Roop Singh Stadium
519643,1798,Pakistan,India,49.4,1,0,Gwalior,Captain Roop Singh Stadium
519644,1798,Pakistan,India,49.5,6,0,Gwalior,Captain Roop Singh Stadium


In [18]:
## Creating a current score column by doing cumulative sum in runs column
# Convert 'runs' column to numeric dtype
df_odi['runs'] = pd.to_numeric(df_odi['runs'], errors='coerce')

# Create a current score column by doing cumulative sum in the 'runs' column
df_odi['current_score'] = df_odi.groupby('match_id')['runs'].cumsum()

/tmp/ipykernel_262268/1956580713.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odi['runs'] = pd.to_numeric(df_odi['runs'], errors='coerce')
/tmp/ipykernel_262268/1956580713.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odi['current_score'] = df_odi.groupby('match_id')['runs'].cumsum()


In [19]:
df_odi

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score
0,0,England,New Zealand,0.1,1,0,Christchurch,AMI Stadium,1
1,0,England,New Zealand,0.2,0,0,Christchurch,AMI Stadium,1
2,0,England,New Zealand,0.3,0,0,Christchurch,AMI Stadium,1
3,0,England,New Zealand,0.4,0,0,Christchurch,AMI Stadium,1
4,0,England,New Zealand,0.5,1,0,Christchurch,AMI Stadium,2
...,...,...,...,...,...,...,...,...,...
519641,1798,Pakistan,India,49.2,1,0,Gwalior,Captain Roop Singh Stadium,245
519642,1798,Pakistan,India,49.3,1,0,Gwalior,Captain Roop Singh Stadium,246
519643,1798,Pakistan,India,49.4,1,0,Gwalior,Captain Roop Singh Stadium,247
519644,1798,Pakistan,India,49.5,6,0,Gwalior,Captain Roop Singh Stadium,253


In [20]:
## making ball_no and over column using ball column
df_odi['over'] = df_odi['ball'].apply(lambda x:str(x).split(".")[0])
df_odi['ball_no'] = df_odi['ball'].apply(lambda x:str(x).split(".")[1])
df_odi

/tmp/ipykernel_262268/4165915864.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odi['over'] = df_odi['ball'].apply(lambda x:str(x).split(".")[0])
/tmp/ipykernel_262268/4165915864.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odi['ball_no'] = df_odi['ball'].apply(lambda x:str(x).split(".")[1])


,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score,over,ball_no
0,0,England,New Zealand,0.1,1,0,Christchurch,AMI Stadium,1,0,1
1,0,England,New Zealand,0.2,0,0,Christchurch,AMI Stadium,1,0,2
2,0,England,New Zealand,0.3,0,0,Christchurch,AMI Stadium,1,0,3
3,0,England,New Zealand,0.4,0,0,Christchurch,AMI Stadium,1,0,4
4,0,England,New Zealand,0.5,1,0,Christchurch,AMI Stadium,2,0,5
...,...,...,...,...,...,...,...,...,...,...,...
519641,1798,Pakistan,India,49.2,1,0,Gwalior,Captain Roop Singh Stadium,245,49,2
519642,1798,Pakistan,India,49.3,1,0,Gwalior,Captain Roop Singh Stadium,246,49,3
519643,1798,Pakistan,India,49.4,1,0,Gwalior,Captain Roop Singh Stadium,247,49,4
519644,1798,Pakistan,India,49.5,6,0,Gwalior,Captain Roop Singh Stadium,253,49,5


In [21]:
## making balls_bowled column using ball_no and over column
df_odi['balls_bowled'] = (df_odi['over'].astype('int')*6) + df_odi['ball_no'].astype('int')
df_odi

/tmp/ipykernel_262268/133229318.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odi['balls_bowled'] = (df_odi['over'].astype('int')*6) + df_odi['ball_no'].astype('int')


,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score,over,ball_no,balls_bowled
0,0,England,New Zealand,0.1,1,0,Christchurch,AMI Stadium,1,0,1,1
1,0,England,New Zealand,0.2,0,0,Christchurch,AMI Stadium,1,0,2,2
2,0,England,New Zealand,0.3,0,0,Christchurch,AMI Stadium,1,0,3,3
3,0,England,New Zealand,0.4,0,0,Christchurch,AMI Stadium,1,0,4,4
4,0,England,New Zealand,0.5,1,0,Christchurch,AMI Stadium,2,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...
519641,1798,Pakistan,India,49.2,1,0,Gwalior,Captain Roop Singh Stadium,245,49,2,296
519642,1798,Pakistan,India,49.3,1,0,Gwalior,Captain Roop Singh Stadium,246,49,3,297
519643,1798,Pakistan,India,49.4,1,0,Gwalior,Captain Roop Singh Stadium,247,49,4,298
519644,1798,Pakistan,India,49.5,6,0,Gwalior,Captain Roop Singh Stadium,253,49,5,299


In [22]:
## making balls_left column
df_odi['balls_left'] = 120 - df_odi['balls_bowled']
df_odi['balls_left'] = df_odi['balls_left'].apply(lambda x:0 if x<0 else x)
df_odi

/tmp/ipykernel_262268/4213441994.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odi['balls_left'] = 120 - df_odi['balls_bowled']
/tmp/ipykernel_262268/4213441994.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odi['balls_left'] = df_odi['balls_left'].apply(lambda x:0 if x<0 else x)


,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score,over,ball_no,balls_bowled,balls_left
0,0,England,New Zealand,0.1,1,0,Christchurch,AMI Stadium,1,0,1,1,119
1,0,England,New Zealand,0.2,0,0,Christchurch,AMI Stadium,1,0,2,2,118
2,0,England,New Zealand,0.3,0,0,Christchurch,AMI Stadium,1,0,3,3,117
3,0,England,New Zealand,0.4,0,0,Christchurch,AMI Stadium,1,0,4,4,116
4,0,England,New Zealand,0.5,1,0,Christchurch,AMI Stadium,2,0,5,5,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...
519641,1798,Pakistan,India,49.2,1,0,Gwalior,Captain Roop Singh Stadium,245,49,2,296,0
519642,1798,Pakistan,India,49.3,1,0,Gwalior,Captain Roop Singh Stadium,246,49,3,297,0
519643,1798,Pakistan,India,49.4,1,0,Gwalior,Captain Roop Singh Stadium,247,49,4,298,0
519644,1798,Pakistan,India,49.5,6,0,Gwalior,Captain Roop Singh Stadium,253,49,5,299,0


In [23]:
## making wickets left using player_dismissed column
df_odi['player_dismissed'] = df_odi['player_dismissed'].astype('int')
df_odi

/tmp/ipykernel_262268/3646018687.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odi['player_dismissed'] = df_odi['player_dismissed'].astype('int')


,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score,over,ball_no,balls_bowled,balls_left
0,0,England,New Zealand,0.1,1,0,Christchurch,AMI Stadium,1,0,1,1,119
1,0,England,New Zealand,0.2,0,0,Christchurch,AMI Stadium,1,0,2,2,118
2,0,England,New Zealand,0.3,0,0,Christchurch,AMI Stadium,1,0,3,3,117
3,0,England,New Zealand,0.4,0,0,Christchurch,AMI Stadium,1,0,4,4,116
4,0,England,New Zealand,0.5,1,0,Christchurch,AMI Stadium,2,0,5,5,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...
519641,1798,Pakistan,India,49.2,1,0,Gwalior,Captain Roop Singh Stadium,245,49,2,296,0
519642,1798,Pakistan,India,49.3,1,0,Gwalior,Captain Roop Singh Stadium,246,49,3,297,0
519643,1798,Pakistan,India,49.4,1,0,Gwalior,Captain Roop Singh Stadium,247,49,4,298,0
519644,1798,Pakistan,India,49.5,6,0,Gwalior,Captain Roop Singh Stadium,253,49,5,299,0


In [24]:
# Group by 'match_id' and calculate cumulative sum
df_odi['wickets_left'] = df_odi.groupby('match_id')['player_dismissed'].cumsum()
df_odi['wickets_left'] = 10 - df_odi['player_dismissed']

/tmp/ipykernel_262268/1193199999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odi['wickets_left'] = df_odi.groupby('match_id')['player_dismissed'].cumsum()
/tmp/ipykernel_262268/1193199999.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odi['wickets_left'] = 10 - df_odi['player_dismissed']


In [25]:
df_odi

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score,over,ball_no,balls_bowled,balls_left,wickets_left
0,0,England,New Zealand,0.1,1,0,Christchurch,AMI Stadium,1,0,1,1,119,10
1,0,England,New Zealand,0.2,0,0,Christchurch,AMI Stadium,1,0,2,2,118,10
2,0,England,New Zealand,0.3,0,0,Christchurch,AMI Stadium,1,0,3,3,117,10
3,0,England,New Zealand,0.4,0,0,Christchurch,AMI Stadium,1,0,4,4,116,10
4,0,England,New Zealand,0.5,1,0,Christchurch,AMI Stadium,2,0,5,5,115,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519641,1798,Pakistan,India,49.2,1,0,Gwalior,Captain Roop Singh Stadium,245,49,2,296,0,10
519642,1798,Pakistan,India,49.3,1,0,Gwalior,Captain Roop Singh Stadium,246,49,3,297,0,10
519643,1798,Pakistan,India,49.4,1,0,Gwalior,Captain Roop Singh Stadium,247,49,4,298,0,10
519644,1798,Pakistan,India,49.5,6,0,Gwalior,Captain Roop Singh Stadium,253,49,5,299,0,10


In [26]:
##Calculating current run rate
df_odi['crr'] = (df_odi['current_score']*6)/df_odi['balls_bowled']

/tmp/ipykernel_262268/45665174.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odi['crr'] = (df_odi['current_score']*6)/df_odi['balls_bowled']


In [27]:
print(df_odi['runs'].unique())
print(df_odi['runs'].isnull().sum())
print(df_odi['runs'].dtype)

[1 0 2 4 3 6 5 7]
0
int64


In [28]:
# Assuming df is your DataFrame containing the data
df_odi['runs'] = pd.to_numeric(df_odi['runs'], errors='coerce')

# Group by 'match_id'
groups = df_odi.groupby('match_id')

# Compute rolling sum for each group with NaN values for the first 29 balls
last_five = []
for group_id, group_data in groups:
    rolling_sum = group_data['runs'].rolling(window=30, min_periods=1).sum()
    # Set NaN for 'last_five_overs_sum' if fewer than 30 balls are available
    if len(group_data) < 30:
        rolling_sum.iloc[-1] = np.nan
    last_five.extend(rolling_sum.values.tolist())

# Assign 'last_five_overs_sum' to the DataFrame
df_odi['last_five'] = last_five

/tmp/ipykernel_262268/3541229983.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odi['runs'] = pd.to_numeric(df_odi['runs'], errors='coerce')
/tmp/ipykernel_262268/3541229983.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_odi['last_five'] = last_five


In [29]:
df_odi

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
0,0,England,New Zealand,0.1,1,0,Christchurch,AMI Stadium,1,0,1,1,119,10,6.000000,1.0
1,0,England,New Zealand,0.2,0,0,Christchurch,AMI Stadium,1,0,2,2,118,10,3.000000,1.0
2,0,England,New Zealand,0.3,0,0,Christchurch,AMI Stadium,1,0,3,3,117,10,2.000000,1.0
3,0,England,New Zealand,0.4,0,0,Christchurch,AMI Stadium,1,0,4,4,116,10,1.500000,1.0
4,0,England,New Zealand,0.5,1,0,Christchurch,AMI Stadium,2,0,5,5,115,10,2.400000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519641,1798,Pakistan,India,49.2,1,0,Gwalior,Captain Roop Singh Stadium,245,49,2,296,0,10,4.966216,36.0
519642,1798,Pakistan,India,49.3,1,0,Gwalior,Captain Roop Singh Stadium,246,49,3,297,0,10,4.969697,36.0
519643,1798,Pakistan,India,49.4,1,0,Gwalior,Captain Roop Singh Stadium,247,49,4,298,0,10,4.973154,36.0
519644,1798,Pakistan,India,49.5,6,0,Gwalior,Captain Roop Singh Stadium,253,49,5,299,0,10,5.076923,42.0


In [34]:
final_df_odi = df_odi.groupby('match_id').sum()['runs'].reset_index().merge(df_odi,on='match_id')
final_df_odi

,match_id,runs_x,batting_team,bowling_team,ball,runs_y,player_dismissed,city,venue,current_score,over,ball_no,balls_bowled,balls_left,wickets_left,crr,last_five
0,0,242,England,New Zealand,0.1,1,0,Christchurch,AMI Stadium,1,0,1,1,119,10,6.000000,1.0
1,0,242,England,New Zealand,0.2,0,0,Christchurch,AMI Stadium,1,0,2,2,118,10,3.000000,1.0
2,0,242,England,New Zealand,0.3,0,0,Christchurch,AMI Stadium,1,0,3,3,117,10,2.000000,1.0
3,0,242,England,New Zealand,0.4,0,0,Christchurch,AMI Stadium,1,0,4,4,116,10,1.500000,1.0
4,0,242,England,New Zealand,0.5,1,0,Christchurch,AMI Stadium,2,0,5,5,115,10,2.400000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377263,1798,255,Pakistan,India,49.2,1,0,Gwalior,Captain Roop Singh Stadium,245,49,2,296,0,10,4.966216,36.0
377264,1798,255,Pakistan,India,49.3,1,0,Gwalior,Captain Roop Singh Stadium,246,49,3,297,0,10,4.969697,36.0
377265,1798,255,Pakistan,India,49.4,1,0,Gwalior,Captain Roop Singh Stadium,247,49,4,298,0,10,4.973154,36.0
377266,1798,255,Pakistan,India,49.5,6,0,Gwalior,Captain Roop Singh Stadium,253,49,5,299,0,10,5.076923,42.0


In [35]:
final_df_odi=final_df_odi[['batting_team','bowling_team','city','current_score','balls_left','wickets_left','crr','last_five','runs_x']]
final_df_odi

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
0,England,New Zealand,Christchurch,1,119,10,6.000000,1.0,242
1,England,New Zealand,Christchurch,1,118,10,3.000000,1.0,242
2,England,New Zealand,Christchurch,1,117,10,2.000000,1.0,242
3,England,New Zealand,Christchurch,1,116,10,1.500000,1.0,242
4,England,New Zealand,Christchurch,2,115,10,2.400000,2.0,242
...,...,...,...,...,...,...,...,...,...
377263,Pakistan,India,Gwalior,245,0,10,4.966216,36.0,255
377264,Pakistan,India,Gwalior,246,0,10,4.969697,36.0,255
377265,Pakistan,India,Gwalior,247,0,10,4.973154,36.0,255
377266,Pakistan,India,Gwalior,253,0,10,5.076923,42.0,255


In [36]:
final_df_odi.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        5
runs_x           0
dtype: int64

In [37]:
final_df_odi.dropna(inplace=True)

/tmp/ipykernel_262268/1409642559.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_odi.dropna(inplace=True)


In [38]:
final_df_odi.isnull().sum()

batting_team     0
bowling_team     0
city             0
current_score    0
balls_left       0
wickets_left     0
crr              0
last_five        0
runs_x           0
dtype: int64

In [39]:
final_df_odi = final_df_odi.sample(final_df_odi.shape[0]) ## shuffeling to neglecting bias

In [40]:
final_df_odi

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
335473,West Indies,Bangladesh,Mirpur,217,0,10,4.536585,26.0,217
267753,Australia,India,Hyderabad,294,0,10,6.485294,46.0,350
377180,Pakistan,India,Gwalior,151,0,10,4.233645,20.0,255
165883,Afghanistan,Pakistan,Leeds,109,0,10,5.109375,27.0,227
237809,India,Pakistan,Mirpur,80,33,10,5.517241,24.0,245
...,...,...,...,...,...,...,...,...,...
14421,Sri Lanka,Australia,Adelaide,156,0,10,4.394366,23.0,274
286406,West Indies,Pakistan,Christchurch,71,28,10,4.630435,25.0,310
355901,New Zealand,Australia,Auckland,130,0,10,5.454545,18.0,238
227361,Pakistan,India,Kolkata,169,0,10,5.173469,17.0,250


In [41]:
##Saving this dataframe final_df_odi in pickle ie level 3_odi 
pickle.dump(final_df_odi,open('dataset_level3_odi.pkl','wb'))